In [10]:
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import nest_asyncio

# nest_asyncio 적용
nest_asyncio.apply()

async def fetch(session, url):
    try:
        async with session.get(url, timeout=aiohttp.ClientTimeout(total=3000)) as response:
            return await response.text()
    except asyncio.TimeoutError:
        print(f"Timeout error for URL: {url}")
        return None

async def get_news_links(session, base_url, page):
    url = f"{base_url}&page={page}"
    response_text = await fetch(session, url)
    if response_text is None:
        return []
    soup = BeautifulSoup(response_text, 'html.parser')
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

async def get_news_content(session, url):
    response_text = await fetch(session, url)
    if response_text is None:
        return None, None
    soup = BeautifulSoup(response_text, 'html.parser')
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article', {'id': 'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    return None, None

async def main():
    today = datetime.today().strftime("%Y%m%d")
    base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today
    async with aiohttp.ClientSession() as session:
        news_links = []
        page = 1
        
        while True:
            links = await get_news_links(session, base_url, page)
            if not links or set(links) & set(news_links):
                break            
            news_links.extend(links)
            page += 1
        
        news_links = set(news_links)
        tasks = [get_news_content(session, link) for link in news_links]
        news_contents = await asyncio.gather(*tasks)
        
        # None 값을 제거
        news_contents = [content for content in news_contents if content[0] is not None]

        # 명사 추출
        nouns_data = []
        for title, content in news_contents:
            nouns_data.append({'Title': title, 'content': content})
        
        df = pd.DataFrame(nouns_data)
        df.to_csv(f"{today}.csv", index=False, encoding='utf-8-sig')

# Jupyter Notebook 환경에서 실행
await main()


In [11]:
today = datetime.today().strftime("%Y%m%d")
df = pd.read_csv(f"{today}.csv",encoding='utf-8')

In [13]:
df

,Title,content
0,韓 기업 발목 잡는 상속세… “자본이득세로 전환해야”,“자본이득세를 도입하면 상속 시점의 과도한 상속세로 발생하는 기업 승계의 어려움을 ...
1,"SK온 ""흑자전환까지 임원연봉 동결"" 비상경영 돌입",C레벨 거취 이사회에 위임·일부 직책은 폐지성과 미흡 시 보임 수시 변경\n\n\n...
2,"`영업이익률 66%` 그 회사…김형태 시프트업 대표, 1조 부호로","2~3일 청약 실시, 11일 상장시총 3.5조 '대형 게임주' 입성\n\n\n\n유..."
3,"[속보] 코스피, 종가 기준 2800선 탈환",사진=연합뉴스코스피지수가 1일 전 거래일 대비 6.49포인트(0.23%) 오른 28...
4,정태영 부회장 “해외 진출 초석 쌓는다”…비자 CEO·글로벌 금융사 CEO들 현대카...,비자 글로벌 CEO·CEMEA 지역 대표들 금융사 최고 경영진 현대카드 깜짝 방문“...
...,...,...
6996,"에코프로비엠, 헝가리공장 1.2조 대출…""유럽사업 박차""",수출입은행 등 통해 1조2000억원 자금 확보헝가리 공장 건설 재원으로 사용 예정내...
6997,"유나이티드문화재단, ‘우남 이승만’ 사진전 개최… 7월 8일까지 무료관람",강남구 유나이티드갤러리 관람 가능\n\n\n\n‘우남 이승만 사진전’이 유나이티드갤...
6998,"메인비즈협회, 디지털협업공장 구축 지원사업 기획기관 선정",한국경영혁신중소기업협회(이하 메인비즈협회)는 중소벤처기업부가 공모한 ‘2024년 디...
6999,"솔루스첨단소재, 차세대 AI 가속기용 동박 양산 돌입",국내 최초 승인…저조도 동박 기술력 입증솔루스첨단소재가 북미 그래픽처리장치(GPU)...
